In [1]:
!pip install -U finance-datareader
!pip install torch torchvision

In [0]:
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import time

In [0]:
#이동평균 칼럼 추가, 당일 포함 
def moveAverage(df,day):
    
    df['MA_{}'.format(day)] = df['Close'].rolling(window=day).mean() 
    #moveAverage(data,30) 30일 이동 평균 추가

In [0]:
#지수 이동 평균
def emoveAverage(df, day):
    df['EMA_{}'.format(day)] = df['Close'].ewm(span=day).mean()


In [0]:
# 이격도(PPO)
def PPO(df, day):
    df['PPO_{}'.format(day)] = (df['Close']/df['MA_{}'.format(day)])*100
    

In [0]:
data = fdr.DataReader('068270',"2019-1-1","2020-5-13")

In [0]:
emoveAverage(data,30)
PPO(data,30)

In [33]:
data['MA_30']


Date
2019-01-02              NaN
2019-01-03              NaN
2019-01-04              NaN
2019-01-07              NaN
2019-01-08              NaN
                  ...      
2020-05-07    204133.333333
2020-05-08    205316.666667
2020-05-11    206200.000000
2020-05-12    207133.333333
2020-05-13    208250.000000
Name: MA_30, Length: 335, dtype: float64

In [34]:
data['EMA_30']

Date
2019-01-02    204879.000000
2019-01-03    202904.816667
2019-01-04    205476.206220
2019-01-07    205706.222327
2019-01-08    205300.496074
                  ...      
2020-05-07    202346.780452
2020-05-08    202872.794617
2020-05-11    203364.872383
2020-05-12    203857.461262
2020-05-13    204576.334729
Name: EMA_30, Length: 335, dtype: float64

In [37]:
data['PPO_30']

Date
2019-01-02           NaN
2019-01-03           NaN
2019-01-04           NaN
2019-01-07           NaN
2019-01-08           NaN
                 ...    
2020-05-07     99.934683
2020-05-08    102.524556
2020-05-11    102.085354
2020-05-12    101.866752
2020-05-13    103.241297
Name: PPO_30, Length: 335, dtype: float64